In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd /content/drive/MyDrive/YLP

/content/drive/MyDrive/YLP


In [ ]:
# Some code in this file is adapted, from:
# Adversarial Example Generation, Inkawhich, N. and Uriegas, E.,  2018, Available from: https://github.com/pytorch/tutorials/blob/master/beginner_source/fgsm_tutorial.py , Accessed: 20 Feb 2022

In [13]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import datetime
import csv
from RKNet import RKNet


# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [14]:
pretrained_model = os.path.join('results',"current_pdenet.pt")
use_cuda=True

In [15]:

shift = transforms.Normalize(
    mean=[0.5,0.5,0.5],
    std=[1.,1.,1.],
  )

# CIFAR-10 Test dataset and dataloader declaration
test_data =  datasets.CIFAR10('../data', train=False, download=True, transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    shift]))

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

# Define device choice
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

Files already downloaded and verified
CUDA Available:  True


In [19]:
# Load model
model = torch.load(pretrained_model)
# Set to eval mode, as no training
model.eval()
# Load to GPU
model = model.to(device)

In [20]:
## Non JB Code Block ##

# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    if epsilon == 0:
      return image
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, -0.5, 0.5)
    # Return the perturbed image
    return perturbed_image

In [21]:
def norm_test( model, device, test_loader, epsilons ):
    start = time.time()
    sStartTime = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    file_name = 'norm_diff_fgsm_resnet' + sStartTime + '.csv'

    csv_path = os.path.join('results',file_name)
    header = ['eps','y0_diff','y_diff']
    with open(csv_path,'w') as f:
      writer = csv.writer(f)
      writer.writerow(header)

        
# v------Non JB Code------v # 
      for eps in epsilons:
  
        # Loop over all examples in test set
        for data, target in test_loader:

            data, target = data.to(device), target.to(device)


            # Set requires_grad attribute of tensor. Important for Attack
            data.requires_grad = True
            
            # Forward pass the data through the model
            output = model(data)
            
            # Calculate the loss
            loss = F.nll_loss(output, target)

            # Zero all existing gradients
            model.zero_grad()

            # Calculate gradients of model in backward pass
            loss.backward()

            # Collect datagrad
            data_grad = data.grad.data

            # Call FGSM Attack
            perturbed_data = fgsm_attack(data, eps, data_grad)

            # Re-classify the perturbed image
            output_tild = model(perturbed_data)
# ^------Non JB Code------^ #

            # Calculate norms and save
            norm_init = torch.linalg.norm(data-perturbed_data)
            norm_final = torch.linalg.norm(output-output_tild)
            writer.writerow([eps,norm_init.item(),norm_final.item()])
            

   

In [22]:
epsilons = [0.001,0.007,0.01,0.07,0.1,0.2]
norm_test(model, device, test_loader, epsilons)